In [1]:
import pyarrow.parquet as pq
import pyarrow as pa
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
# The predicted period x days
period = 3

In [3]:
air_visit_df = pd.read_csv('input/air_visit_data.csv')
sample_submission_df = pd.read_csv('input/sample_submission.csv')

In [4]:
feature_df = sample_submission_df.copy()
feature_df['feature1'] = pd.Series(np.random.randn(len(sample_submission_df)), index=sample_submission_df.index)
feature_df['feature2'] = pd.Series(np.random.randn(len(sample_submission_df)), index=sample_submission_df.index)

In [5]:
def predict(*x):
    return np.random.randn()

In [22]:
def prev_date_str(sdate, dlt=1):
    return  dt.datetime.strftime(dt.datetime.strptime(sdate, '%Y-%m-%d') - dt.timedelta(dlt), '%Y-%m-%d')

def ext_air_store_id(asidd):
    return '_'.join(asidd.split('_')[:2])

def ext_date(asidd):
    return asidd.split('_')[-1]

def air_store_id_index(df):
    asid_df = df['id'].apply(lambda x: ext_air_store_id(x)).to_frame()
    tdf = asid_df.groupby('id').count()
    return tdf.index

def predicted_date(df):
    visit_date = df['id'].apply(lambda x: ext_date(x)).to_frame()
    return (visit_date.min().get(0), visit_date.max().get(0), visit_date['id'])

In [23]:
first_date, last_date, predicted_dates = predicted_date(feature_df)

In [12]:
base_df = pd.DataFrame(air_store_id_index(feature_df))

In [26]:
for dlt in range(period, 0, -1):
    target_date = prev_date_str(first_date, dlt)
    base_df = pd.merge(base_df, air_visit_df[air_visit_df['visit_date'] ==  target_date][['air_store_id', 'visitors']], how='left', on='air_store_id')
    base_df.columns = pd.Index(np.append(base_df.columns[:-1].values, target_date))

KeyError: 'air_store_id'

In [ ]:
for d in predicted_dates.tolist():
    base_df[d] = pd.Series(np.nan, index=base_df.index)

In [ ]:
feature_df[:1]

In [ ]:
base_df[:1]

In [ ]:
for idx, row in feature_df.iterrows():
    asid = ext_air_store_id(row['id'])
    date = ext_date(row['id'])
    condf = pd.concat(row.to_frame(), base_df[base_df['air_store_id'] == asid][prev_date_str(date, i) for i in  range(period, 0, -1)])
    print(condf.as_matrix)
    break

In [ ]:
base_df[base_df['air_store_id'] == 'air_00a91d42b08b08d9'].as_matrix()

In [ ]:
sample_feature_df

In [ ]:
base_df